In [1]:
import pandas as pd
import numpy as np

In [2]:
# importing all the relevent excel files.

order_table = pd.read_excel('Company X - Order Report.xlsx')
pincode_table = pd.read_excel('Company X - Pincode Zones.xlsx')
skumaster_table = pd.read_excel('Company X - SKU Master.xlsx')
invoice_table = pd.read_excel('Courier Company - Invoice.xlsx')
courier_rates_table = pd.read_excel('Courier Company - Rates.xlsx')

In [3]:
order_table.head()

,ExternOrderNo,SKU,Order Qty
0,2001827036,8904223818706,1.0
1,2001827036,8904223819093,1.0
2,2001827036,8904223819109,1.0
3,2001827036,8904223818430,1.0
4,2001827036,8904223819277,1.0


In [4]:
# pincode_table.head()

In [5]:
skumaster_table.head()

,SKU,Weight (g)
0,8904223815682,210
1,8904223815859,165
2,8904223815866,113
3,8904223815873,65
4,8904223816214,120


In [6]:
# invoice_table.head()

In [7]:
courier_rates_table.head()

,Zone,Weight Slabs,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge
0,A,0.25,29.5,23.6,13.6,23.6
1,B,0.50,33.0,28.3,20.5,28.3
2,C,0.75,40.1,38.9,31.9,38.9
3,D,1.25,45.4,44.8,41.3,44.8
4,E,1.50,56.6,55.5,50.7,55.5


## Capitalizing the Zone column in pincode table and invoice table.

In [8]:
pincode_table['Zone'] = pincode_table['Zone'].apply(lambda x: x.capitalize())
invoice_table['Zone']= invoice_table['Zone'].apply(lambda x: x.capitalize())

In [9]:
pincode_table.head()

,Warehouse Pincode,Customer Pincode,Zone
0,121003,507101,D
1,121003,486886,D
2,121003,532484,D
3,121003,143001,B
4,121003,515591,D


In [10]:
invoice_table.head()

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.)
0,1091117222124,2001806232,1.30,121003,507101,D,Forward charges,135.0
1,1091117222194,2001806273,1.00,121003,486886,D,Forward charges,90.2
2,1091117222931,2001806408,2.50,121003,532484,D,Forward charges,224.6
3,1091117223244,2001806458,1.00,121003,143001,B,Forward charges,61.3
4,1091117229345,2001807012,0.15,121003,515591,D,Forward charges,45.4


## Ensuring the data type is in correct data type


In [11]:
# order_table.dtypes

In [12]:
# pincode_table.dtypes

In [13]:
# skumaster_table.dtypes

In [14]:
# invoice_table.dtypes

In [15]:
# courier_rates_table.dtypes

## Create a new dataframe which will have the information as per X.


In [16]:
x_table = pd.merge(left= order_table, right= skumaster_table, left_on= 'SKU', right_on='SKU',
                  how='inner')

In [17]:
# x_table

In [18]:
x_table['total weight (KG)'] = round((x_table['Order Qty']*x_table['Weight (g)'])/1000,2)

In [19]:
# x_table

In [20]:
x_table = x_table[['ExternOrderNo', 'total weight (KG)']].groupby(by='ExternOrderNo').sum()

In [21]:
x_table.reset_index(inplace=True)

In [22]:
x_table =pd.merge(left= x_table, right= invoice_table[['Order ID','Customer Pincode', 'Type of Shipment']], left_on= 'ExternOrderNo', right_on='Order ID', how='inner')

In [23]:
x_table.drop(columns='Order ID', inplace=True)

In [24]:
# x_table

In [25]:
x_table= pd.merge(left=x_table, right=pincode_table[['Customer Pincode', 'Zone']], left_on='Customer Pincode',
         right_on='Customer Pincode', how='inner')

In [26]:
# x_table

In [27]:
x_table.duplicated().sum()

50

In [28]:
x_table.drop_duplicates(inplace=True)

In [29]:
x_table = pd.merge(left=x_table, right=courier_rates_table, left_on='Zone', right_on='Zone', how='inner')

In [30]:
# x_table

## Defining a function to get the applicable weight

In [31]:
def smallest_multiple(weight_slab, total_weight):
    remainder = total_weight % weight_slab
    if remainder == 0:
        return total_weight
    else:
        return total_weight + weight_slab - remainder

In [32]:
x_table['applicable weight']= x_table.apply(lambda row: smallest_multiple(row['Weight Slabs'], row['total weight (KG)']), axis=1)

In [33]:
# x_table

## Defining a function to get the expected price


In [34]:
def expected_amt(shipment, total_weight, weight_slab, frd_charges, frd_extra_charges, rto_charges, rto_extra_charges):
    if shipment=='Forward charges':
        x= total_weight/weight_slab
        y = x-(x-1)
        
        fixed_charge= y*frd_charges
        
        z= x-1
        additional_charges = z*frd_extra_charges
        
        total_amount= fixed_charge + additional_charges
        
        return total_amount
    
    else:
        x= total_weight/weight_slab
        y = x-(x-1)
        
        fixed_charge= y * (rto_charges+frd_charges)
        
        z= x-1
        additional_charges= z*(frd_extra_charges+ rto_extra_charges)
        
        total_amount= fixed_charge+additional_charges
        
        return total_amount

In [35]:
x_table['expected Price']= x_table.apply(lambda row: expected_amt(row['Type of Shipment'], row['applicable weight'], row['Weight Slabs'],
                      row['Forward Fixed Charge'], row['Forward Additional Weight Slab Charge'], row['RTO Fixed Charge'],
                      row['RTO Additional Weight Slab Charge']), axis=1)

In [36]:
# x_table

In [37]:
x_table['Weight Slabs']= x_table['Weight Slabs'].apply(lambda x: '%.2f'%x)

In [38]:
# x_table

In [39]:
x_table.rename(columns={'ExternOrderNo': 'Order ID', 'total weight (KG)':'Total weight as per X (KG)', 'Zone':
               'Delivery Zone as per X', 'Weight Slabs': 'Weight slab as per X (KG)', 'expected Price':'Expected Charge as per X (Rs.)'},
              inplace=True)

In [40]:
# x_table

## Creating dataframe for Courier Company info.

In [41]:
courier_table= pd.merge(left=invoice_table[['AWB Code', 'Order ID', 'Charged Weight', 'Zone', 'Billing Amount (Rs.)']], right=courier_rates_table
         [['Zone', 'Weight Slabs']], left_on='Zone', right_on='Zone', how='inner')

In [42]:
# courier_table

In [43]:
courier_table.rename(columns={'Charged Weight':'Total weight as per the Courier Company (KG)', 'Zone': 'Delivery Zone charged by Courier Company', 
                             'Billing Amount (Rs.)': 'Charges Billed by Courier Company (Rs.)', 'Weight Slabs':'Weight slab charged by Courier Company (KG)'}, inplace=True)

## Creating a comparison dataframe

In [44]:
comparison_table =pd.merge(left= x_table[['Order ID', 'Total weight as per X (KG)', 'Delivery Zone as per X', 'Weight slab as per X (KG)', 
               'Expected Charge as per X (Rs.)']], right= courier_table, left_on= 'Order ID', right_on='Order ID', how='inner')

In [45]:
# comparison_table

In [46]:
comparison_table= comparison_table.iloc[:,[0, 5, 1, 3,6, 9, 2, 7, 4, 8]]

In [47]:
comparison_table['Difference Between Expected Charges and Billed Charged (Rs.)']= round((comparison_table['Expected Charge as per X (Rs.)']-
                                             comparison_table['Charges Billed by Courier Company (Rs.)']),2)

In [48]:
comparison_table

,Order ID,AWB Code,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per the Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Expected Charge as per X (Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charged (Rs.)
0,2001806210,1091117221940,0.22,0.50,2.92,0.50,B,B,33.0,174.5,-141.5
1,2001806233,1091117222135,0.25,0.50,0.78,0.50,B,B,33.0,61.3,-28.3
2,2001806304,1091117222360,0.50,0.50,0.71,1.25,B,D,33.0,90.2,-57.2
3,2001810549,1091117806263,0.98,0.50,1.86,1.25,B,D,61.3,179.8,-118.5
4,2001811306,1091117958163,0.94,0.50,1.10,1.25,B,D,61.3,135.0,-73.7
...,...,...,...,...,...,...,...,...,...,...,...
119,2001811475,1091121844806,0.69,1.50,0.50,0.50,E,B,56.6,33.0,23.6
120,2001811604,1091118004245,0.72,1.50,0.80,0.50,E,B,56.6,61.3,-4.7
121,2001818390,1091120014461,0.84,1.50,0.80,1.50,E,E,107.3,213.5,-106.2
122,2001819252,1091120352712,0.27,1.50,0.30,0.50,E,B,56.6,33.0,23.6


## Creating a Summary Table

In [49]:
summary_table = comparison_table.copy()

In [50]:
def category_(x):
    if x>0:
        return 'Total Orders where X has been undercharged'
    elif x<0:
        return 'Total Orders where X has been overcharged'
    else:
        return 'Total Orders where X has been correctly charged'

In [51]:
summary_table['Category'] = summary_table['Difference Between Expected Charges and Billed Charged (Rs.)'].apply(lambda x:category_(x))

In [52]:
summary_table

,Order ID,AWB Code,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per the Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Expected Charge as per X (Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charged (Rs.),Category
0,2001806210,1091117221940,0.22,0.50,2.92,0.50,B,B,33.0,174.5,-141.5,Total Orders where X has been overcharged
1,2001806233,1091117222135,0.25,0.50,0.78,0.50,B,B,33.0,61.3,-28.3,Total Orders where X has been overcharged
2,2001806304,1091117222360,0.50,0.50,0.71,1.25,B,D,33.0,90.2,-57.2,Total Orders where X has been overcharged
3,2001810549,1091117806263,0.98,0.50,1.86,1.25,B,D,61.3,179.8,-118.5,Total Orders where X has been overcharged
4,2001811306,1091117958163,0.94,0.50,1.10,1.25,B,D,61.3,135.0,-73.7,Total Orders where X has been overcharged
...,...,...,...,...,...,...,...,...,...,...,...,...
119,2001811475,1091121844806,0.69,1.50,0.50,0.50,E,B,56.6,33.0,23.6,Total Orders where X has been undercharged
120,2001811604,1091118004245,0.72,1.50,0.80,0.50,E,B,56.6,61.3,-4.7,Total Orders where X has been overcharged
121,2001818390,1091120014461,0.84,1.50,0.80,1.50,E,E,107.3,213.5,-106.2,Total Orders where X has been overcharged
122,2001819252,1091120352712,0.27,1.50,0.30,0.50,E,B,56.6,33.0,23.6,Total Orders where X has been undercharged


In [53]:
def amt(x, y):
    if y>0:
        return y
    elif y<0:
        return (-y)
    else:
        return x

In [54]:
summary_table['Amount (Rs.)']= summary_table.apply(lambda row: amt(row['Charges Billed by Courier Company (Rs.)'],
                            row['Difference Between Expected Charges and Billed Charged (Rs.)']), axis=1)

In [55]:
summary_table= summary_table.groupby(by='Category').agg({'Category': 'count', 'Amount (Rs.)': 'sum'})

In [56]:
# summary_table

In [57]:
summary_table.rename(columns={'Category': 'Count'}, inplace=True)

In [58]:
summary_table.index.name = ' '

In [59]:
summary_table

,Count,Amount (Rs.)
,,
Total Orders where X has been correctly charged,14,881.3
Total Orders where X has been overcharged,101,6439.5
Total Orders where X has been undercharged,9,182.4


In [60]:
summary_table.to_excel('Summary Table.xlsx', index=True)

In [61]:
comparison_table.to_excel('Comparison Table.xlsx', index=False)